# Descripcion del procesamiento de la base de datos 

In [1]:
import pandas as pd

from pathlib import Path

In [2]:
# definir las rutas del archivo 
# El uso de pathlib deja con referencias realtivas las carpetas para que no sea necesario referencialas a nivel local 
base_path = Path().resolve().parent # Obtiene la ruta absoluta del directorio actual



In [3]:
folder = base_path / "input"  # Subcarpeta 'csv' dentro del directorio actual
excel_file = folder / "bd_risk_model.xlsx"  # Archivo dentro de la subcarpeta


In [4]:
# Cargar el archivo Excel
excel_data = pd.ExcelFile(excel_file)

# Obtener los nombres de las hojas (sheets)
sheet_names = excel_data.sheet_names

# Mostrar los nombres de los sheets
print("Nombres de los sheets:", sheet_names)


Nombres de los sheets: ['comercial_troncal', 'gobierno', 'comercial_secundario ', 'internacional', 'comercial_regional', 'aerotaxis', 'comercial carga', 'especial_carga']


In [5]:
sheet_names

['comercial_troncal',
 'gobierno',
 'comercial_secundario ',
 'internacional',
 'comercial_regional',
 'aerotaxis',
 'comercial carga',
 'especial_carga']

In [6]:
# Por falta de datos no se procesaran los datos de especial_carga
sheet_names.remove("especial_carga")

### Limpieza y Procesamiento 

Resolver problema de las celdas duplicadas 

In [11]:
# Lista para almacenar los DataFrames
dataframes_list = []

# Leer cada sheet y guardarlo como DataFrame en la lista
for sheet_name in sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name , header= None)
    print(f"Hoja {sheet_name} leída y almacenada como DataFrame.")
    dataframes_list.append(df)

# Ahora puedes procesar la lista de DataFrames
print(f"Se han leído {len(dataframes_list)} hojas en total.")

Hoja comercial_troncal leída y almacenada como DataFrame.
Hoja gobierno leída y almacenada como DataFrame.
Hoja comercial_secundario  leída y almacenada como DataFrame.
Hoja internacional leída y almacenada como DataFrame.
Hoja comercial_regional leída y almacenada como DataFrame.
Hoja aerotaxis leída y almacenada como DataFrame.
Hoja comercial carga leída y almacenada como DataFrame.
Se han leído 7 hojas en total.


###  LIMPIEZA DATAFRAME

In [39]:
def procesar_dataframe(df):
    """
    Procesa un DataFrame que contiene celdas combinadas y múltiples encabezados,
    transformándolo en un formato limpio y estructurado con columnas separadas por
    Empresa, Año y Rubros.

    ### Parámetros:
    - df (pd.DataFrame): DataFrame original cargado desde un archivo Excel, que contiene:
        - Encabezados combinados con nombres de empresas y años.
        - Rubros (como "ACTIVO CORRIENTE", "ACTIVO FIJO", etc.) en la primera columna.
    
    ### Entradas:
    - DataFrame con celdas combinadas en las filas 0 y 1 que contienen información de
      empresa y año, y datos tabulares en las filas siguientes.

    ### Salidas:
    - pd.DataFrame: Un nuevo DataFrame estructurado con las siguientes columnas:
        - Empresa: Nombre de la empresa extraído del encabezado.
        - Año: Año extraído del encabezado.
        - Rubros: Variables originales como "ACTIVO CORRIENTE", "ACTIVO FIJO", etc.
        - Valores: Datos correspondientes a cada rubro para cada empresa y año.

    ### Pasos realizados:
    1. Rellena celdas combinadas con el nombre de la empresa.
    2. Crea encabezados limpios combinando Empresa y Año.
    3. Elimina las filas usadas como encabezado.
    4. Convierte el DataFrame al formato largo (long format) para facilitar la manipulación.
    5. Separa Empresa y Año en columnas distintas.
    6. Rearma el DataFrame en formato ancho (wide format) para presentar cada rubro como una columna.
    7. Limpia nombres de columnas eliminando espacios innecesarios.

    ### Retorno:
    - df_final (pd.DataFrame): DataFrame procesado con la estructura limpia mencionada.
    """
    # Paso 1: Rellenar las celdas combinadas
    df.iloc[0] = df.iloc[0].fillna(method='ffill')

    # Paso 2: Crear un encabezado limpio combinando Empresa y Año
    new_headers = []
    for i in range(len(df.columns)):
        if i == 0:  # La primera columna suele ser la de los rubros (e.g., "ACTIVO CORRIENTE")
            new_headers.append("Rubros")
        else:
            new_headers.append(f"{df.iloc[0, i]} - {df.iloc[1, i]}")

    df.columns = new_headers

    # Paso 3: Eliminar filas de encabezados antiguos
    df_cleaned = df.iloc[2:].reset_index(drop=True)

    # Paso 4: Convertir los datos a un formato largo (melt)
    df_long = pd.melt(
        df_cleaned, 
        id_vars=["Rubros"], 
        var_name="Empresa_Año", 
        value_name="Valor"
    )

    # Paso 5: Separar "Empresa" y "Año" en columnas distintas
    df_long[["Empresa", "Año"]] = df_long["Empresa_Año"].str.rsplit(" - ", n=1, expand=True)
    df_long = df_long.drop_duplicates(subset=["Empresa", "Año", "Rubros"])
    

    # Paso 6: Pivotar para que las variables estén en columnas separadas
    df_final = df_long.pivot(index=["Empresa", "Año"], columns="Rubros", values="Valor").reset_index()

    # Paso 7: Renombrar columnas para facilitar su lectura
    df_final.columns.name = None  # Eliminar el nombre del índice generado por pivot
    df_final.rename(columns=lambda x: x.strip(), inplace=True)  # Eliminar espacios en los nombres de columnas

    return df_final

In [40]:
sheet_names

['comercial_troncal',
 'gobierno',
 'comercial_secundario ',
 'internacional',
 'comercial_regional',
 'aerotaxis',
 'comercial carga']

In [41]:

 # Lista de DataFrames a procesar = dataframes_list
# original_names = sheet_names  # Nombres originales de los DataFrames

# Diccionario para almacenar los DataFrames procesados con sus nuevos nombres
processed_dfs = {}

# Iterar sobre los DataFrames y sus nombres originales
for df, name in zip(dataframes_list, sheet_names):
    processed_name = f"{name}_processed"  # Crear el nuevo nombre
    processed_dfs[processed_name] = procesar_dataframe(df)  # Procesar y guardar en el diccionario

# Acceder a los DataFrames procesados
for key, value in processed_dfs.items():
    print(f"Nombre: {key}")
    print(value.head())  # Mostrar los primeros datos procesados

C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[0] = df.iloc[0].fillna(method='ffill')
C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[0] = df.iloc[0].fillna(method='ffill')
C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[0] = df.iloc[0].fillna(method='ffill')
C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[0] = df.iloc[0].fillna(method='ffill')
C:\U

Nombre: comercial_troncal_processed
                                             Empresa     Año ACTIVO CORRIENTE  \
0                                     FAST COLOMBIA     2019        250447708   
1                                     FAST COLOMBIA     2020        123488532   
2                                     FAST COLOMBIA   2020.0      123488532.0   
3                                     FAST COLOMBIA   2021.0      307236280.0   
4  4- EMPRESA AEREA DE SERVICIOS Y FACILITACIÓN L...    2022     208554442000   

    ACTIVO FIJO COSTOS Y GASTOS OPERACIONALES INGRESOS OPERACIONALES OTROS  \
0    1506115534                     894237424              765247109   NaN   
1    1529341118                     718934555              410841381   NaN   
2  1529341118.0                   718934555.0            410841381.0   NaN   
3  2704849600.0                   972390576.0           1021983263.0   NaN   
4  485117377000                  462781900000           493702641000   NaN   

  OTROS 

C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[0] = df.iloc[0].fillna(method='ffill')
C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AEROSUCRE S.A.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[0] = df.iloc[0].fillna(method='ffill')
C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AIR COLOMBIA LTDA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[0] = df.iloc[0].fillna(method='ffill')
C:\Users\andre\AppData\Local\Temp\ipykernel_17692\989326232.py:36: Fut

In [43]:
processed_dfs.keys()

dict_keys(['comercial_troncal_processed', 'gobierno_processed', 'comercial_secundario _processed', 'internacional_processed', 'comercial_regional_processed', 'aerotaxis_processed', 'comercial carga_processed'])

### Estandarizacion de Nombres 

In [60]:
### Estandarizar Nombres de las empresas 

def obtener_nombres_empresas_unicos(diccionario_dataframes):
    """
    Toma un diccionario con DataFrames como valores, encuentra los nombres únicos
    de las empresas en cada DataFrame y retorna un diccionario con los mismos keys
    pero los valores como listas de nombres únicos.

    ### Parámetros:
    - diccionario_dataframes (dict): Diccionario donde:
        - Claves: Identificadores de cada DataFrame.
        - Valores: DataFrames que contienen una columna llamada "Empresa" (o equivalente).

    ### Retorno:
    - dict: Un diccionario con los mismos keys pero con listas de nombres únicos de empresas como valores.
    """
    nombres_empresas_unicos = {}

    for key, df in diccionario_dataframes.items():
        if "Empresa" in df.columns:
            # Obtener nombres únicos de la columna "Empresa"
            empresas_unicas = df["Empresa"].dropna().unique().tolist()
            nombres_empresas_unicos[key] = empresas_unicas
        else:
            # Si no existe la columna "Empresa", retorna lista vacía
            nombres_empresas_unicos[key] = []

    return nombres_empresas_unicos

In [61]:
# Llamar la función para obtener nombres unicos de las empresas 

dict_nombres = obtener_nombres_empresas_unicos(processed_dfs)


In [62]:
# Obtener llaves 

dict_nombres.keys()

dict_keys(['comercial_troncal_processed', 'gobierno_processed', 'comercial_secundario _processed', 'internacional_processed', 'comercial_regional_processed', 'aerotaxis_processed', 'comercial carga_processed'])

In [63]:
dict_nombres['comercial carga_processed']

[' LAS - LINEAS AEREAS SURAMERICANAS',
 'AEROLINEA DEL CARIBE',
 'AEROLÍNEA DEL CARIBE S.A. - AER CARIBE S.A.',
 'AEROLÍNEAS ANDINAS S.A. - ALIANSA',
 'AEROSUCRE',
 'AEROSUCRE S.A.',
 'AIR COLOMBIA LTDA',
 'ALIANSA',
 'ALIANSA S.A.- AEROLINEAS ANDINAS S.A.',
 'CV.CARGO',
 'CV.CARGO su',
 'LANCO',
 'LANCO - LINEAS AEREA DE COLOMBIA',
 'LAS',
 'LAS S.A. - LINEAS AEREAS SURAMERICANAS S.A.',
 'LINEA AEREA CARGUERA DE COLOMBIA S.A.( PAQ.DE LANCO)',
 'SADELCA LTDA',
 'SADELCA LTDA (suspendida)',
 'SELVA LTDA',
 'SELVA LTDA (suspendida)',
 'TAMPA',
 'TAMPA  S.A.']

In [64]:
dict_reemplazo_comercial_troncal = {
    "AEROREPUBLICA - COPA AIRLINE COLOMBIA -": "COPA_AIRLINES",
    "COPA AIRLINE COLOMBIA - AEROREPUBLICA": "COPA_AIRLINES",
    "VIVA COLOMBIA": "VIVA AIR",
    "AIRES": "LATAM",
}


In [65]:
dict_reemplazo_comercial_secundario= {
    "FAST COLOMBIA  - VIVA": "VIVA AIR",
    "ADA": "ADA",
    "ADA AEROLINEA DE ANTIOQUIA": "ADA",
    "AEROREPUBLICA": "COPA_AIRLINES",
    "AIRES": "LATAM",
    "AVIANCA ": "AVIANCA",
    "EASY FLY": "EASYFLY",
    "EASYFLY": "EASYFLY",
    "MOON FLIGHTS S.A.S. ": "MOON FLIGHTS",
    "REGIONAL EXPRESS AMERICA ": "REGIONAL EXPRESS",
    "SEARCA": "SEARCA",
    "TAC": "TAC"
}

In [66]:
dict_reemplazo_comercial_regional = {
    "AEXPA": "AEXPA",
    "AEXPA ": "AEXPA",
    "AEXPA SUSPENDIDA\nN/A": "AEXPA",
    "SEARCA S.A": "SEARCA",
    "TAC - TRANSPORTE AEREO DE COLOMBIA S.A.": "TAC"
}

In [67]:
dict_reemplazo_carga =  {
    'LAS - LINEAS AEREAS SURAMERICANAS': 'LAS',
    'AEROLINEA DEL CARIBE': 'AER CARIBE',
    'AEROLÍNEA DEL CARIBE S.A. - AER CARIBE S.A.': 'AER CARIBE',
    'AEROLÍNEAS ANDINAS S.A. - ALIANSA': 'ALIANSA',
    'AEROSUCRE': 'AEROSUCRE',
    'AEROSUCRE S.A.': 'AEROSUCRE',
    'AIR COLOMBIA LTDA': 'AIR COLOMBIA',
    'ALIANSA': 'ALIANSA',
    'ALIANSA S.A.- AEROLINEAS ANDINAS S.A.': 'ALIANSA',
    'CV.CARGO': 'CV CARGO',
    'CV.CARGO su': 'CV CARGO',
    'LANCO': 'LANCO',
    'LANCO - LINEAS AEREA DE COLOMBIA': 'LANCO',
    'LAS': 'LAS',
    'LAS S.A. - LINEAS AEREAS SURAMERICANAS S.A.': 'LAS',
    'LINEA AEREA CARGUERA DE COLOMBIA S.A.( PAQ.DE LANCO)': 'LANCO',
    'SADELCA LTDA': 'SADELCA',
    'SADELCA LTDA (suspendida)': 'SADELCA',
    'SELVA LTDA': 'SELVA',
    'SELVA LTDA (suspendida)': 'SELVA',
    'TAMPA': 'TAMPA',
    'TAMPA  S.A.': 'TAMPA'
}

In [68]:
dict_reemplazo_aerotaxis = {
    'AEROVIAS REGIONALES DEL ORIENTE S.A.S ARO S.A.S': 'AEROVIAS REGIONALES DEL ORIENTE SAS',
    '1- AEROLINEAS DEL LLANO ALLA SAS': 'AEROLINEAS DEL LLANO SAS',
    '10- HELISERVICE SAS\nNIT 800053544-4': 'HELISERVICE SAS',
    '11- SERVICIOS AEREOS DEL GUAVIARE LTDA\nNIT 800152647-9': 'SERVICIOS AEREOS DEL GUAVIARE LTDA',
    '12- SOCIEDAD AERONAUTICA DE SANTANDER SAS\nNIT: 800.043.763-8': 'SOCIEDAD AERONAUTICA DE SANTANDER SAS',
    '13- SERVICIO AEREO DE LA ORINOQUIA SAS\nNIT 901527740-3': 'SERVICIO AEREO DE LA ORINOQUIA SAS',
    '14- ISATECH CORPORATION SAS\nNIT 900292243-1': 'ISATECH CORPORATION SAS',
    '15- HELIJET SAS\nNIT: 830108998-1': 'HELIJET SAS',
    '16- HELIGOLFO SAS': 'HELIGOLFO SAS',
    '17- HELICENTRO SAS': 'HELICENTRO SAS',
    '18- HANGAR 29 SAS\nNIT: 900713438-7': 'HANGAR 29 SAS',
    '19- CHARTER DEL CARIBE SAS': 'CHARTER DEL CARIBE SAS',
    '2- LLANERA DE AVIACIÓN SAS': 'LLANERA DE AVIACION SAS',
    '20- ALLIANCE AVIATION COLOMBIA SAS': 'ALLIANCE AVIATION COLOMBIA SAS',
    '21- RIO SUR SAS\nNIT: 800149580-3': 'RIO SUR SAS',
    '22- AERO TAXI GUAYMARAL ATG SAS\nNIT 900142183-X': 'AERO TAXI GUAYMARAL SAS',
    '23- AVIANLINE CHAERTERS SAS\nNIT: 901076189-8': 'AVIANLINE CHAERTERS SAS',
    '24- AVIONES DEL CESAR SAS': 'AVIONES DEL CESAR SAS',
    '25- AEROLINEAS LLANERAS ARALL LTDA\nNIT 892001725 - 2': 'AEROLINEAS LLANERAS ARALL LTDA',
    '26- AEROEJECUTIVOS DE ANTIOQUIA SAS\nNIT: 811030546-1': 'AEROEJECUTIVOS DE ANTIOQUIA SAS',
    '27- PACIFICA DE AVIACION SAS\nNIT 830,514,084-6': 'PACIFICA DE AVIACION SAS',
    '28- AVIATEC SAS': 'AVIATEC SAS',
    '29- INTERNACIONAL EJECUTIVA DE AVIACIÓN SAS': 'INTERNACIONAL EJECUTIVA DE AVIACION SAS',
    '3- NATIV AIR SAS': 'NATIV AIR SAS',
    '30- SYNERJET LATINA': 'SYNERJET LATINA',
    '31- Taxi Aereo Colombiano TAERCO': 'TAXI AEREO COLOMBIANO',
    '32- GOOD FLY CO SAS\nNIT: 900638251-6': 'GOOD FLY CO SAS',
    '33- SOLAIR SAS\nNIT 805019730': 'SOLAIR SAS',
    '34- HELICOPTEROS AGRICOLAS DE COLOMBIA SAS': 'HELICOPTEROS AGRICOLAS DE COLOMBIA SAS',
    '35- HELICE LTDA': 'HELICE LTDA',
    '36- SERVICIOS AEREOS PANAMERICANOS SAS': 'SERVICIOS AEREOS PANAMERICANOS SAS',
    '37- AEROTAXI DEL UPIA SAS\nNIT 800213471-2': 'AEROTAXI DEL UPIA SAS',
    '38- SERVICIO AÉREO DEL ORIENTE S.A.S. - SAO S.A.S. Nit. 822,001,727-2': 'SERVICIO AEREO DEL ORIENTE SAS',
    '39- AEROLINEA REGIONAL DE COLOMBIA SAS\nNIT 800115731-2': 'AEROLINEA REGIONAL DE COLOMBIA SAS',
    '4- AEROVIAS REGIONALES DEL ORIENTE SAS\nNIT 800086107 - 0': 'AEROVIAS REGIONALES DEL ORIENTE SAS',
    '40- AEROESTAR LTDA\nNIT 900132478-8': 'AEROESTAR LTDA',
    '41- HELICÓPTEROS NACIONALES DE COLOMBIA SAS EN REORGANIZACIÓN- HELICOL SAS': 'HELICOL SAS',
    '42- HELICES DEL META SAS\nNIT 892.002.838 - 0': 'HELICES DEL META SAS',
    '43- Internacional Ejecutiva de Aviación SAS': 'INTERNACIONAL EJECUTIVA DE AVIACION SAS',
    '44- LINEAS AEREAS DEL NORTE DE SANTANDER LANS SAS': 'LINEAS AEREAS DEL NORTE DE SANTANDER SAS',
    '45- SERVICIOS INTEGRALES HELICOPORTADOS S.A.S SICHER\nNIT: 800.055.561-9': 'SERVICIOS INTEGRALES HELICOPORTADOS SAS',
    '46- SERVICIOS AEREOS ESPECIALIZADOS EN TRANSPORTES PETROLEROS\nNIT: 860.075.359': 'SERVICIOS AEREOS ESPECIALIZADOS EN TRANSPORTES PETROLEROS',
    '47- SIS SOLUCIONES INTEGRALES GNSS S.A.S\nNIT: 830.126.135-9': 'SOLUCIONES INTEGRALES GNSS SAS',
    '5- AEROEXPRESS SAS\nNIT: 811033782': 'AEROEXPRESS SAS',
    '6- SERVICIO AEREO DE CAPURGANA SA': 'SERVICIO AEREO DE CAPURGANA SA',
    '7- DELTA HELICOPTEROS SAS\nNIT: 900226716': 'DELTA HELICOPTEROS SAS',
    '8- Helistar SAS\nNIT: 811020344-6': 'HELISTAR SAS',
    '9- AEROVIAS REGIONALES DEL ORIENTE SAS\nNIT: 800086107 - 0': 'AEROVIAS REGIONALES DEL ORIENTE SAS',
}


In [69]:
dict_reemplazo_aerotaxis.update({
    'ACA AEROCHARTER ANDINA S.A.': 'AEROCHARTER ANDINA SA',
    'ACA-  AEROCHARTER ANDINA S.A.': 'AEROCHARTER ANDINA SA',
    'AER CARIBE - AEROLINEA DEL CARIBE': 'AEROLINEA DEL CARIBE',
    'AERO EXPRESS SAS': 'AEROEXPRESS SAS',
    'AERO TAXI GUAYMARAL': 'AERO TAXI GUAYMARAL SAS',
    'AEROAPOYO LTDA - TRANSPORTE AEREO DEL APOYO PETROLERO': 'AEROAPOYO LTDA',
    'AEROCOL LTDA': 'AEROCOL LTDA',
    'AEROEJECUTIVOS DE ANTIOQUIA ': 'AEROEJECUTIVOS DE ANTIOQUIA SAS',
    'AEROEJECUTIVOS DE ANTIOQUIA S.A.': 'AEROEJECUTIVOS DE ANTIOQUIA SAS',
    'AEROESTAR LTDA ': 'AEROESTAR LTDA',
    'AEROEXPRESS S.A.': 'AEROEXPRESS SAS',
    'AEROGALAN LTDA': 'AEROGALAN LTDA',
    'AEROGALAN LTDA AHORA STAR BLUE': 'AEROGALAN LTDA',
    'AEROLINEAS ALAS DE COLOMBIA': 'AEROLINEAS ALAS DE COLOMBIA',
    'AEROLINEAS ALAS DE COLOMBIA\nN/A': 'AEROLINEAS ALAS DE COLOMBIA',
    'AEROLINEAS ALAS DE COLOMBIA (SUSPENDIDA)\nN/A': 'AEROLINEAS ALAS DE COLOMBIA',
    'AEROLINEAS DEL LLANO S.A.S': 'AEROLINEAS DEL LLANO SAS',
    'AEROLINEAS DEL LLANO SAS ALLAS': 'AEROLINEAS DEL LLANO SAS',
    'AEROLINEAS LLANERAS ARALL LTDA ': 'AEROLINEAS LLANERAS ARALL LTDA',
    'AEROLINEAS PETROLERAS SAS ALPES ': 'AEROLINEAS PETROLERAS SAS',
    'AEROMEL \n(CANCELADA)': 'AEROMEL',
    'AEROMEL \nCANCELADA': 'AEROMEL',
    'AEROMEL su': 'AEROMEL',
    'AEROMENEGUA': 'AEROMENEGUA',
    'AEROPACA SAS': 'AEROPACA SAS',
    'AEROSERVICIOS ESPECIALIZADO SAS ASES SAS': 'AEROSERVICIOS ESPECIALIZADO SAS',
    'AEROTAXI DEL UPIA SAS': 'AEROTAXI DEL UPIA SAS',
    'AEROTAXI GUAYMARAL': 'AERO TAXI GUAYMARAL SAS',
    'AERUPIA': 'AERUPIA',
})

In [70]:
dict_reemplazo_aerotaxis.update({
    'AIR SIGNATURE': 'AIR SIGNATURE',
    'AIR SIGNATURE (CANCELADA)': 'AIR SIGNATURE',
    'AIR SIGNATURE - ': 'AIR SIGNATURE',
    'AIR SIGNATURE - \nSU': 'AIR SIGNATURE',
    'AIR SIGNATURE - PEOPER VENCIDO': 'AIR SIGNATURE',
    'ALLAS - AEROVANGUARDIA LTDA': 'AEROVANGUARDIA LTDA',
    'ALPES EU -AEROLINEAS PETROLERAS DE CASANARE E.U.': 'AEROLINEAS PETROLERAS DE CASANARE EU',
    'AMERICAS AIR SAS': 'AMERICAS AIR SAS',
    'ARALL - AEROLINEAS LLANERAS': 'AEROLINEAS LLANERAS ARALL LTDA',
    'ARALL - AEROLINEAS LLANERAS (No se toma en cuenta por mala calidad de la información)': 'AEROLINEAS LLANERAS ARALL LTDA',
    'ARO LTDA - AEROVIAS REGIONALES DEL ORIENTE': 'AEROVIAS REGIONALES DEL ORIENTE LTDA',
    'ASES - AEROSERVICIOS ESPECIALIZADOS \n': 'AEROSERVICIOS ESPECIALIZADOS SAS',
    'ASES - AEROSERVICIOS ESPECIALIZADOS \nPEOPER VENCIDO': 'AEROSERVICIOS ESPECIALIZADOS SAS',
    'AVIACION ESPECIALIZADA HANGAR 29': 'HANGAR 29 SAS',
    'AVIANLINE\n': 'AVIANLINE',
    'AVIANLINE CHARTER´S SAS\n': 'AVIANLINE CHARTERS SAS',
    'AVIOCESAR LTDA': 'AVIOCESAR LTDA',
    'AVIOCHARTER': 'AVIOCHARTER',
    'AVIOCHARTER ': 'AVIOCHARTER',
    'AVIOCHARTER (no presento información)': 'AVIOCHARTER',
    'AVIOINGENIERIA': 'AVIOINGENIERIA',
})

In [71]:
dict_reemplazo_aerotaxis.update({
    'CENTRAL CHARTER DE COLOMBIA S.A.': 'CENTRAL CHARTER DE COLOMBIA SA',
    'CHARTER DEL CARIBE S.A.': 'CHARTER DEL CARIBE SAS',
    'CHARTER DEL CARIBE S.A.S': 'CHARTER DEL CARIBE SAS',
    'CHARTER EXPRESS SAS': 'CHARTER EXPRESS SAS',
    'CUSIANA\n   PEOPER VENCIDO': 'CUSIANA',
    'CUSIANA\nSU\n  ': 'CUSIANA',
    'CUSIANA (SUSPENDIDA)\n\n  ': 'CUSIANA',
    'CUSTOM AVIATION SAS ANTES HELIFLY': 'CUSTOM AVIATION SAS',
    'DELTA HELICOPTEROS ': 'DELTA HELICOPTEROS SAS',
    'DELTA HELICOPTEROS (SERVICIO POR TODO LO ALTO)': 'DELTA HELICOPTEROS SAS',
    'DELTA HELICÓPTEROS S.A.S': 'DELTA HELICOPTEROS SAS',
    'GOOD FLY': 'GOOD FLY SAS',
    'HELI JET SAS': 'HELIJET SAS',
    'HELIAV LTDA  - HELICOPEROS Y AVIONES LTDA': 'HELIAV LTDA',
    'HELIAV S.A.S  - HELICOPEROS Y AVIONES S.A.S': 'HELIAV SAS',
    'HELICOL S.A.': 'HELICOL SAS',
    'HELICOL S.A.S': 'HELICOL SAS',
    'HELIFLY': 'HELIFLY SAS',
    'HELIGOLFO S.A. ': 'HELIGOLFO SAS',
    'HELISERVICE LTDA (SERVICIOS AEREOS)': 'HELISERVICE SAS',
    'HELISERVICE SAS': 'HELISERVICE SAS',
    'HELISTAR LTDA': 'HELISTAR SAS',
    'HELISUR ERVICE LTDA (SERVICIOS AEREOS)': 'HELISUR SAS',
    'HELISUR SAS': 'HELISUR SAS',
    'HELITEC S.A.(HELICOPTEROS TERRITORIALES DE COLOMBIA)': 'HELITEC SAS',
    'HELITEC S.A.(HELICOPTEROS TERRITORIALES DE COLOMBIA) (SUSPENDIDA)': 'HELITEC SAS',
})

In [72]:
dict_reemplazo_aerotaxis.update({
    'HORIZONTAL DE AVIACION': 'HORIZONTAL DE AVIACION SAS',
    'HORIZONTAL DE AVIACIÓN S.A.S. en reorganización FLEXAIR S.A.S': 'HORIZONTAL DE AVIACION SAS',
    'INTEREJECUTIVA ': 'INTERNACIONAL EJECUTIVA DE AVIACION SAS',
    'INTEREJECUTIVA-  LA OXCI - AEROVILLA': 'INTERNACIONAL EJECUTIVA DE AVIACION SAS',
    'INTERNACIONAL EJECUTIVA DE AVIACION S.A.S.\n“INTEREJECUTIVA”': 'INTERNACIONAL EJECUTIVA DE AVIACION SAS',
    'LANS - LINEAS AEREAS DEL NORTE DE SANTANDER LTDA': 'LINEAS AEREAS DEL NORTE DE SANTANDER SAS',
    'LANS S.A.S. LINEAS AEREAS DEL NORTE DE SANTANDER S.A.S.': 'LINEAS AEREAS DEL NORTE DE SANTANDER SAS',
    'LASER AÉREO': 'LASER AEREO SAS',
    'LATINOAMERICANA DE SERVICIOS AEREO S.A.S. LASER AEREO S.A.S.': 'LASER AEREO SAS',
    'LLANERA DE AVIACION S.A.': 'LLANERA DE AVIACION SAS',
    'LOS HALCONES SAS': 'LOS HALCONES SAS',
    'NACIONAL DE AVIACION': 'NACIONAL DE AVIACION SAS',
    'NACIONAL DE AVIACION\nN/A': 'NACIONAL DE AVIACION SAS',
    'NACIONAL DE AVIACION\nN/A (SUSPENDIDA)': 'NACIONAL DE AVIACION SAS',
    'NATIV AIR S.A.S. antes  "HELIAV S.A.S."': 'NATIV AIR SAS',
    'PACIFICA DE AVIACION': 'PACIFICA DE AVIACION SAS',
    'RIO SUR S.A.': 'RIO SUR SAS',
    'SADI (SOCIEDAD AEREA DE IBAGUE LTDA)': 'SOCIEDAD AEREA DE IBAGUE LTDA',
    'SAE SERVICIOS AÉREOS ESPECIALES GLOBAL LIFE AMBULANCIAS S.A.S.': 'SERVICIOS AEREOS ESPECIALES SAS',
    'SAER LTDA (SERVICIO AEREO REGIONAL )': 'SERVICIO AEREO REGIONAL LTDA',
    'SAER LTDA (SERVICIO AEREO REGIONAL ) ': 'SERVICIO AEREO REGIONAL LTDA',
    'SAER LTDA (SERVICIO AEREO REGIONAL ) (No presento información)': 'SERVICIO AEREO REGIONAL LTDA',
    'SARPA\nSERVICIOS AEREOS PANAMERICANOS S.A.S.': 'SERVICIOS AEREOS PANAMERICANOS SAS',
    'SARPA LTDA (SERVICIOS AEREOS PANAMERICANOS LTDA)': 'SERVICIOS AEREOS PANAMERICANOS SAS',
    'SASA Colombia S.A.  SOCIEDAD AERONAUTICA DE  SANTANDER': 'SOCIEDAD AERONAUTICA DE SANTANDER SAS',
    'SAVIARE LTDA  (SERVICIOS AEREOS DEL GUAVIARE LTDA': 'SERVICIOS AEREOS DEL GUAVIARE LTDA',
    'SERVICIOS INTEGRALES HELICOPORTADOS S.A.S. -SICHER HELICOPTERS S.A.S ': 'SICHER HELICOPTERS SAS',
    'SICHER HELICOPTERS S.A.S': 'SICHER HELICOPTERS SAS',
    'SICHER HELICOPTERS S.A.S ': 'SICHER HELICOPTERS SAS',
    'SIS SOLUCIONES INTEGRALES GNSS': 'SOLUCIONES INTEGRALES GNSS SAS',
    'SIS SOLUCIONES INTEGRALES GNSS S.A.S ': 'SOLUCIONES INTEGRALES GNSS SAS',
    'SOLAIR S.A.S.': 'SOLAIR SAS',
    'STARBLUE AIRLINES - ANTES AEROGALAN - LINEAS AEREAS GALAN SAS': 'STARBLUE AIRLINES SAS',
})

In [73]:
dict_reemplazo_aerotaxis.update({
    'TACA - TAXI AEREO CARIBEÑO': 'TAXI AEREO CARIBEÑO LTDA',
    'TACA - TAXI AEREO CARIBEÑO (SUSPENDIDO)': 'TAXI AEREO CARIBEÑO LTDA',
    'TACA LTDA - TAXI AEREO CARIBEÑO LTDA': 'TAXI AEREO CARIBEÑO LTDA',
    'TAERCO - TAXI AEREO COLOMBIANO LTDA.': 'TAXI AEREO COLOMBIANO LTDA',
    'TAERCO LTDA  - TAXI AEREO COLOMBIANO LTDA': 'TAXI AEREO COLOMBIANO LTDA',
    'TARI LTDA - TRANSPORTES AEREOS DEL ARIARI': 'TRANSPORTES AEREOS DEL ARIARI LTDA',
    'TAXCO S.A.S': 'TAXCO SAS',
    'TAXCO S.A.S- TAXI AEREO DE LA COSTA': 'TAXCO SAS',
    'TECNIAEREAS LTDA': 'TECNIAEREAS LTDA',
    'VERTICAL DE AVIACION LTDA': 'VERTICAL DE AVIACION LTDA',
    'VIANA LTDA': 'VIANA LTDA',
    'VIANA S.A.S': 'VIANA SAS',
    'VIP HELICOPTEROS\nTECNIAEREAS LTDA': 'VIP HELICOPTEROS LTDA',
    'VIP HELICOPTEROS\nTECNIAEREAS LTDA (No presento información)': 'VIP HELICOPTEROS LTDA',
    'VIP HELICOPTEROS\nTECNIAEREAS LTDA (suspendido)': 'VIP HELICOPTEROS LTDA',
})

### Aplicar de forma iterativa los diccionarios de reemplazo de valores 

In [74]:
def aplicar_reemplazos(dict_dataframes, dict_reemplazos):
    """
    Aplica los reemplazos definidos en diccionarios a los DataFrames especificados.
    
    Args:
        dict_dataframes (dict): Diccionario donde las keys son nombres o identificadores
                                y los valores son DataFrames.
        dict_reemplazos (dict): Diccionario donde las keys coinciden con las keys de dict_dataframes
                                y los valores son los diccionarios de reemplazo a aplicar.

    Returns:
        dict: Diccionario con los DataFrames actualizados después de aplicar los reemplazos.
    """
    dict_resultado = {}

    for key, df in dict_dataframes.items():
        # Verificar si hay un diccionario de reemplazo para este DataFrame
        if key in dict_reemplazos:
            reemplazos = dict_reemplazos[key]
            # Aplicar los reemplazos al DataFrame
            df_actualizado = df.replace(reemplazos)
            dict_resultado[key] = df_actualizado
        else:
            # Si no hay reemplazos, mantener el DataFrame sin cambios
            dict_resultado[key] = df

    return dict_resultado

In [76]:
processed_dfs.keys()

dict_keys(['comercial_troncal_processed', 'gobierno_processed', 'comercial_secundario _processed', 'internacional_processed', 'comercial_regional_processed', 'aerotaxis_processed', 'comercial carga_processed'])

In [77]:
diccionario_reemplazos = {'comercial_troncal_processed':dict_reemplazo_comercial_troncal,
                          'comercial_secundario _processed':dict_reemplazo_comercial_secundario,
                          'comercial_regional_processed':dict_reemplazo_comercial_regional,
                          'aerotaxis_processed':dict_reemplazo_aerotaxis,
                          'comercial carga_processed':dict_reemplazo_carga
    
}

In [78]:
dataframes_valores_unicos = aplicar_reemplazos(processed_dfs, diccionario_reemplazos)


In [79]:
dataframes_valores_unicos.keys()

dict_keys(['comercial_troncal_processed', 'gobierno_processed', 'comercial_secundario _processed', 'internacional_processed', 'comercial_regional_processed', 'aerotaxis_processed', 'comercial carga_processed'])

In [82]:
def agregar_caracteristica(dict_dataframes, caracteristicas_map):
    """
    Agrega una columna de característica a los DataFrames según un mapeo definido.

    Args:
        dict_dataframes (dict): Diccionario donde las keys son identificadores
                                y los valores son DataFrames.
        caracteristicas_map (dict): Diccionario donde las keys son identificadores de los DataFrames
                                     y los valores son las características a asignar.

    Returns:
        dict: Diccionario con los DataFrames actualizados después de agregar la columna de característica.
    """
    dict_resultado = {}

    for key, df in dict_dataframes.items():
        # Verificar si hay una característica asignada para este DataFrame
        if key in caracteristicas_map:
            caracteristica = caracteristicas_map[key]
            # Agregar la columna al DataFrame
            df_actualizado = df.copy()
            df_actualizado['Caracteristica'] = caracteristica
            dict_resultado[key] = df_actualizado
        else:
            # Si no hay característica asignada, mantener el DataFrame sin cambios
            dict_resultado[key] = df

    return dict_resultado

In [83]:
caracteristicas_map = {
    'comercial_troncal_processed': 'comercial',
    'comercial_secundario _processed': 'comercial',
    'comercial_regional_processed': 'comercial',
    'comercial carga_processed': 'carga',
    'internacional_processed': 'comercial',
    'gobierno_processed': 'gobierno',
    'aerotaxis_processed': 'aerotaxi'
}

In [87]:
dataframes_paso_3 = agregar_caracteristica(dataframes_valores_unicos, caracteristicas_map)


In [89]:
dataframes_paso_3.keys()

dict_keys(['comercial_troncal_processed', 'gobierno_processed', 'comercial_secundario _processed', 'internacional_processed', 'comercial_regional_processed', 'aerotaxis_processed', 'comercial carga_processed'])

In [91]:
### Se eliminan los internacionales debido a que no se cuenta con suficiente informacion 

# Eliminar clave internacional_processed
if 'internacional_processed' in dataframes_paso_3.keys():
    del dataframes_paso_3['internacional_processed']


In [93]:
dataframes_paso_3.keys()

dict_keys(['comercial_troncal_processed', 'gobierno_processed', 'comercial_secundario _processed', 'comercial_regional_processed', 'aerotaxis_processed', 'comercial carga_processed'])

### Creacion del Dataframe Consolidado 

In [105]:
def consolidar_dataframes(dict_dataframes, columnas_eliminar):
    """
    Consolida varios DataFrames en uno solo, elimina duplicados y formatea las columnas.

    Args:
        dict_dataframes (dict): Diccionario donde las keys son identificadores y los valores son DataFrames.
        columnas_eliminar (list): Lista de nombres de columnas a eliminar del DataFrame consolidado.

    Returns:
        pd.DataFrame: DataFrame consolidado, sin duplicados y con columnas formateadas.
    """
    # Lista para almacenar los DataFrames consolidados
    dataframes_list = []

    for key, df in dict_dataframes.items():
        # Convertir columnas que no sean 'Año' ni 'Caracteristica' al formato millar
        df_formateado = df.copy()
        columnas_formatear = [col for col in df.columns if col not in ['Año', 'Caracteristica','Empresa']]
        for col in columnas_formatear:
            df_formateado[col] = pd.to_numeric(df_formateado[col], errors='coerce').fillna(0).astype(int)
        
        # Añadir el DataFrame formateado a la lista
        dataframes_list.append(df_formateado)

    # Concatenar todos los DataFrames en uno solo
    consolidado = pd.concat(dataframes_list, ignore_index=True)

    # Eliminar duplicados (mismo nombre de empresa y año)
    consolidado = consolidado.drop_duplicates(subset=['Empresa', 'Año'])

    # Eliminar las columnas especificadas
    consolidado = consolidado.drop(columns=columnas_eliminar, errors='ignore')

    return consolidado

In [130]:
dataframes_paso_3['comercial_troncal_processed'].columns

Index(['Empresa', 'Año', 'ACTIVO CORRIENTE', 'ACTIVO FIJO',
       'COSTOS Y GASTOS OPERACIONALES', 'INGRESOS OPERACIONALES', 'OTROS',
       'OTROS ACTIVOS', 'OTROS GASTOS', 'OTROS INGRESOS',
       'PASIVO A LARGO PLAZO', 'PASIVO CORRIENTE',
       'PATRIMONIO_CAPITAL(PAGADO)', 'PROVISION PARA IMPUESTOS',
       'TOTAL GASTOS', 'TOTAL INGRESOS', 'TOTAL PASIVO',
       'TOTAL PASIVO Y PATRIMONIO', 'TOTAL PATRIMONIO',
       'UTILIDAD (PERDIDA) DEL EJERCICIO', 'UTILIDAD (PERDIDA) NETA',
       'UTILIDAD (PERDIDA) OPERACIONAL', 'UTILIDAD (PERDIDAS) ACUMULADAS',
       'Caracteristica'],
      dtype='object')

In [131]:
lists_columnas_eliminar = ['UTILIDAD (PERDIDA) DEL EJERCICIO','UTILIDAD (PERDIDAS) ACUMULADAS','TOTAL PASIVO','TOTAL PASIVO Y PATRIMONIO','A  C  T  I  V  O  S', 'BALANCE GENERAL', 'G A S T O S',
       'I N G R E S O S', 'INGRESOS Y EGRESOS', 'P  A  S  I  V  O  S',
       'P A T R I M O N I O','TOTAL ACTIVO','TOTAL GASTOS','TOTAL INGRESOS']

In [138]:
df_consolidado = consolidar_dataframes(dataframes_paso_3, lists_columnas_eliminar)

In [139]:
df_consolidado.columns

Index(['Empresa', 'Año', 'ACTIVO CORRIENTE', 'ACTIVO FIJO',
       'COSTOS Y GASTOS OPERACIONALES', 'INGRESOS OPERACIONALES', 'OTROS',
       'OTROS ACTIVOS', 'OTROS GASTOS', 'OTROS INGRESOS',
       'PASIVO A LARGO PLAZO', 'PASIVO CORRIENTE',
       'PATRIMONIO_CAPITAL(PAGADO)', 'PROVISION PARA IMPUESTOS',
       'TOTAL PATRIMONIO', 'UTILIDAD (PERDIDA) NETA',
       'UTILIDAD (PERDIDA) OPERACIONAL', 'Caracteristica', 'CAPITAL (PAGADO)',
       'OTROS CAPITAL', 'OTROS PASIVOS', 'ACTIVOS CORRIENTES'],
      dtype='object')

In [140]:
df_consolidado

,Empresa,Año,ACTIVO CORRIENTE,ACTIVO FIJO,COSTOS Y GASTOS OPERACIONALES,INGRESOS OPERACIONALES,OTROS,OTROS ACTIVOS,OTROS GASTOS,OTROS INGRESOS,...,PATRIMONIO_CAPITAL(PAGADO),PROVISION PARA IMPUESTOS,TOTAL PATRIMONIO,UTILIDAD (PERDIDA) NETA,UTILIDAD (PERDIDA) OPERACIONAL,Caracteristica,CAPITAL (PAGADO),OTROS CAPITAL,OTROS PASIVOS,ACTIVOS CORRIENTES
0,FAST COLOMBIA,2019,2.504477e+08,1506115534,894237424,765247109,0,0,0,0,...,8.210270e+06,38923731,2.605942e+07,-163921892,-128990315,comercial,NaN,NaN,NaN,NaN
1,FAST COLOMBIA,2020,1.234885e+08,1529341118,718934555,410841381,0,0,0,0,...,8.210270e+06,0,-3.495184e+08,-398843699,-308093174,comercial,NaN,NaN,NaN,NaN
2,FAST COLOMBIA,2020.0,1.234885e+08,1529341118,718934555,410841381,0,0,0,0,...,8.210270e+06,0,-3.495184e+08,-398843699,-308093174,comercial,NaN,NaN,NaN,NaN
3,FAST COLOMBIA,2021.0,3.072363e+08,-2147483648,972390576,1021983263,0,0,0,0,...,8.210270e+06,57984306,-3.994280e+08,6402712,49592687,comercial,NaN,NaN,NaN,NaN
4,4- EMPRESA AEREA DE SERVICIOS Y FACILITACIÓN L...,2022,-2.147484e+09,-2147483648,-2147483648,-2147483648,0,0,-2147483648,-2147483648,...,-2.147484e+09,-2147483648,-2.147484e+09,-2147483648,-2147483648,comercial,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,TAMPA,2016,NaN,1472529000,978858000,1177487000,187395000,371469000,51181000,23333000,...,NaN,27181000,7.224220e+08,143600000,198629000,carga,180982000.0,NaN,0.0,335587000.0
1333,TAMPA,2016.0,NaN,1472529000,978858000,1177487000,187395000,371469000,51181000,23333000,...,NaN,27181000,7.224220e+08,143600000,198629000,carga,180982000.0,NaN,0.0,335587000.0
1334,TAMPA,2017,NaN,1314657064,933718867,1049737310,240164148,408098750,51242445,26438850,...,NaN,32091893,7.767482e+08,59122955,116018443,carga,180981800.0,NaN,0.0,460479618.0
1335,TAMPA,2017.0,NaN,1314657064,933718867,1049737310,184368894,406277144,51242445,26438850,...,NaN,32091893,7.767484e+08,59122955,116018443,carga,180982000.0,NaN,0.0,462301224.0


In [141]:
df_consolidado.columns

Index(['Empresa', 'Año', 'ACTIVO CORRIENTE', 'ACTIVO FIJO',
       'COSTOS Y GASTOS OPERACIONALES', 'INGRESOS OPERACIONALES', 'OTROS',
       'OTROS ACTIVOS', 'OTROS GASTOS', 'OTROS INGRESOS',
       'PASIVO A LARGO PLAZO', 'PASIVO CORRIENTE',
       'PATRIMONIO_CAPITAL(PAGADO)', 'PROVISION PARA IMPUESTOS',
       'TOTAL PATRIMONIO', 'UTILIDAD (PERDIDA) NETA',
       'UTILIDAD (PERDIDA) OPERACIONAL', 'Caracteristica', 'CAPITAL (PAGADO)',
       'OTROS CAPITAL', 'OTROS PASIVOS', 'ACTIVOS CORRIENTES'],
      dtype='object')

In [149]:
df_consolidado.columns

Index(['Empresa', 'Año', 'ACTIVO CORRIENTE', 'ACTIVO FIJO',
       'COSTOS Y GASTOS OPERACIONALES', 'INGRESOS OPERACIONALES', 'OTROS',
       'OTROS ACTIVOS', 'OTROS GASTOS', 'OTROS INGRESOS',
       'PASIVO A LARGO PLAZO', 'PASIVO CORRIENTE',
       'PATRIMONIO_CAPITAL(PAGADO)', 'PROVISION PARA IMPUESTOS',
       'TOTAL PATRIMONIO', 'UTILIDAD (PERDIDA) NETA',
       'UTILIDAD (PERDIDA) OPERACIONAL', 'Caracteristica', 'CAPITAL (PAGADO)',
       'OTROS CAPITAL', 'OTROS PASIVOS', 'ACTIVOS CORRIENTES'],
      dtype='object')

In [150]:
df_consolidado['ACTIVO CORRIENTE'] = df_consolidado['ACTIVO CORRIENTE'].fillna(0)
df_consolidado['ACTIVOS CORRIENTES'] = df_consolidado['ACTIVOS CORRIENTES'].fillna(0)

In [151]:
# Crear una nueva columna que sea la suma de ambas
df_consolidado['ACTIVOS_CORRIENTES_TOTAL'] = (
    df_consolidado['ACTIVO CORRIENTE'] + df_consolidado['ACTIVOS CORRIENTES']
)

# Eliminar una de las columnas originales (por ejemplo, 'ACTIVOS CORRIENTES')
df_consolidado = df_consolidado.drop(columns=['ACTIVOS CORRIENTES' , 'ACTIVO CORRIENTE'])

In [154]:
#renombra la columna 

df_consolidado.rename(columns={'ACTIVOS_CORRIENTES_TOTAL': 'ACTIVO CORRIENTE'}, inplace=True)

In [155]:
df_consolidado

,Empresa,Año,ACTIVO FIJO,COSTOS Y GASTOS OPERACIONALES,INGRESOS OPERACIONALES,OTROS,OTROS ACTIVOS,OTROS GASTOS,OTROS INGRESOS,PASIVO A LARGO PLAZO,...,PATRIMONIO_CAPITAL(PAGADO),PROVISION PARA IMPUESTOS,TOTAL PATRIMONIO,UTILIDAD (PERDIDA) NETA,UTILIDAD (PERDIDA) OPERACIONAL,Caracteristica,CAPITAL (PAGADO),OTROS CAPITAL,OTROS PASIVOS,ACTIVO CORRIENTE
0,FAST COLOMBIA,2019,1506115534,894237424,765247109,0,0,0,0,1300928679,...,8.210270e+06,38923731,2.605942e+07,-163921892,-128990315,comercial,NaN,NaN,NaN,2.504477e+08
1,FAST COLOMBIA,2020,1529341118,718934555,410841381,0,0,0,0,1643112114,...,8.210270e+06,0,-3.495184e+08,-398843699,-308093174,comercial,NaN,NaN,NaN,1.234885e+08
2,FAST COLOMBIA,2020.0,1529341118,718934555,410841381,0,0,0,0,1643112114,...,8.210270e+06,0,-3.495184e+08,-398843699,-308093174,comercial,NaN,NaN,NaN,1.234885e+08
3,FAST COLOMBIA,2021.0,-2147483648,972390576,1021983263,0,0,0,0,-2147483648,...,8.210270e+06,57984306,-3.994280e+08,6402712,49592687,comercial,NaN,NaN,NaN,3.072363e+08
4,4- EMPRESA AEREA DE SERVICIOS Y FACILITACIÓN L...,2022,-2147483648,-2147483648,-2147483648,0,0,-2147483648,-2147483648,-2147483648,...,-2.147484e+09,-2147483648,-2.147484e+09,-2147483648,-2147483648,comercial,NaN,NaN,NaN,-2.147484e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,TAMPA,2016,1472529000,978858000,1177487000,187395000,371469000,51181000,23333000,851424000,...,NaN,27181000,7.224220e+08,143600000,198629000,carga,180982000.0,NaN,0.0,1.006761e+09
1333,TAMPA,2016.0,1472529000,978858000,1177487000,187395000,371469000,51181000,23333000,851424000,...,NaN,27181000,7.224220e+08,143600000,198629000,carga,180982000.0,NaN,0.0,1.006761e+09
1334,TAMPA,2017,1314657064,933718867,1049737310,240164148,408098750,51242445,26438850,758864483,...,NaN,32091893,7.767482e+08,59122955,116018443,carga,180981800.0,NaN,0.0,1.381439e+09
1335,TAMPA,2017.0,1314657064,933718867,1049737310,184368894,406277144,51242445,26438850,758864483,...,NaN,32091893,7.767484e+08,59122955,116018443,carga,180982000.0,NaN,0.0,1.386904e+09


In [157]:
df_consolidado['CAPITAL (PAGADO)'] = df_consolidado['CAPITAL (PAGADO)'].fillna(0)
df_consolidado['PATRIMONIO_CAPITAL(PAGADO)'] = df_consolidado['PATRIMONIO_CAPITAL(PAGADO)'].fillna(0)

# Crear una nueva columna que sea la suma de ambas
df_consolidado['CAPITAL SOCIAL'] = (
    df_consolidado['CAPITAL (PAGADO)'] + df_consolidado['PATRIMONIO_CAPITAL(PAGADO)']
)

# Eliminar una de las columnas originales (por ejemplo, 'ACTIVOS CORRIENTES')
df_consolidado = df_consolidado.drop(columns=['CAPITAL (PAGADO)' , 'PATRIMONIO_CAPITAL(PAGADO)'])

In [159]:
df_consolidado.columns

Index(['Empresa', 'Año', 'ACTIVO FIJO', 'COSTOS Y GASTOS OPERACIONALES',
       'INGRESOS OPERACIONALES', 'OTROS', 'OTROS ACTIVOS', 'OTROS GASTOS',
       'OTROS INGRESOS', 'PASIVO A LARGO PLAZO', 'PASIVO CORRIENTE',
       'PROVISION PARA IMPUESTOS', 'TOTAL PATRIMONIO',
       'UTILIDAD (PERDIDA) NETA', 'UTILIDAD (PERDIDA) OPERACIONAL',
       'Caracteristica', 'OTROS CAPITAL', 'OTROS PASIVOS', 'ACTIVO CORRIENTE',
       'CAPITAL SOCIAL'],
      dtype='object')

In [163]:
# Llenar los NaN con 0
df_consolidado.fillna(0, inplace=True)

# Renombrar las columnas
df_consolidado.rename(columns={'OTROS': 'OTROS PATRIMONIO', 'Caracteristica': 'TIPO_EMPRESA'}, inplace=True)

# Reorganizar el orden de las columnas
column_order = [
    'Empresa', 'Año',
    'ACTIVO FIJO', 'ACTIVO CORRIENTE', 'OTROS ACTIVOS',  # ACTIVOS
    'PASIVO A LARGO PLAZO', 'PASIVO CORRIENTE', 'OTROS PASIVOS',  # PASIVOS
    'CAPITAL SOCIAL', 'OTROS PATRIMONIO', 'TOTAL PATRIMONIO',  # PATRIMONIO
    'INGRESOS OPERACIONALES', 'OTROS INGRESOS',  # INGRESOS
    'COSTOS Y GASTOS OPERACIONALES', 'OTROS GASTOS',  # GASTOS
    'UTILIDAD (PERDIDA) OPERACIONAL', 'PROVISION PARA IMPUESTOS', 'UTILIDAD (PERDIDA) NETA',  # UTILIDADES
    'TIPO_EMPRESA'  # Otros
]

# Reorganizar las columnas del DataFrame
df_consolidado = df_consolidado[column_order]


C:\Users\andre\AppData\Local\Temp\ipykernel_17692\100066350.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado.fillna(0, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_17692\100066350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado.rename(columns={'OTROS': 'OTROS PATRIMONIO', 'Caracteristica': 'TIPO_EMPRESA'}, inplace=True)


In [164]:
df_consolidado

,Empresa,Año,ACTIVO FIJO,ACTIVO CORRIENTE,OTROS ACTIVOS,PASIVO A LARGO PLAZO,PASIVO CORRIENTE,OTROS PASIVOS,CAPITAL SOCIAL,OTROS PATRIMONIO,TOTAL PATRIMONIO,INGRESOS OPERACIONALES,OTROS INGRESOS,COSTOS Y GASTOS OPERACIONALES,OTROS GASTOS,UTILIDAD (PERDIDA) OPERACIONAL,PROVISION PARA IMPUESTOS,UTILIDAD (PERDIDA) NETA,TIPO_EMPRESA
0,FAST COLOMBIA,2019,1506115534,2.504477e+08,0,1300928679,429575146,0.0,8.210270e+06,0,2.605942e+07,765247109,0,894237424,0,-128990315,38923731,-163921892,comercial
1,FAST COLOMBIA,2020,1529341118,1.234885e+08,0,1643112114,359235930,0.0,8.210270e+06,0,-3.495184e+08,410841381,0,718934555,0,-308093174,0,-398843699,comercial
2,FAST COLOMBIA,2020.0,1529341118,1.234885e+08,0,1643112114,359235930,0.0,8.210270e+06,0,-3.495184e+08,410841381,0,718934555,0,-308093174,0,-398843699,comercial
3,FAST COLOMBIA,2021.0,-2147483648,3.072363e+08,0,-2147483648,679566639,0.0,8.210270e+06,0,-3.994280e+08,1021983263,0,972390576,0,49592687,57984306,6402712,comercial
4,4- EMPRESA AEREA DE SERVICIOS Y FACILITACIÓN L...,2022,-2147483648,-2.147484e+09,0,-2147483648,-2147483648,0.0,-2.147484e+09,0,-2.147484e+09,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,comercial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,TAMPA,2016,1472529000,1.006761e+09,371469000,851424000,605739000,0.0,1.809820e+08,187395000,7.224220e+08,1177487000,23333000,978858000,51181000,198629000,27181000,143600000,carga
1333,TAMPA,2016.0,1472529000,1.006761e+09,371469000,851424000,605739000,0.0,1.809820e+08,187395000,7.224220e+08,1177487000,23333000,978858000,51181000,198629000,27181000,143600000,carga
1334,TAMPA,2017,1314657064,1.381439e+09,408098750,758864483,647622717,0.0,1.809818e+08,240164148,7.767482e+08,1049737310,26438850,933718867,51242445,116018443,32091893,59122955,carga
1335,TAMPA,2017.0,1314657064,1.386904e+09,406277144,758864483,647622717,0.0,1.809820e+08,184368894,7.767484e+08,1049737310,26438850,933718867,51242445,116018443,32091893,59122955,carga


In [165]:
# Convertir los valores de la columna TIPO_EMPRESA a mayúsculas
df_consolidado['TIPO_EMPRESA'] = df_consolidado['TIPO_EMPRESA'].str.upper()

In [166]:
df_consolidado

,Empresa,Año,ACTIVO FIJO,ACTIVO CORRIENTE,OTROS ACTIVOS,PASIVO A LARGO PLAZO,PASIVO CORRIENTE,OTROS PASIVOS,CAPITAL SOCIAL,OTROS PATRIMONIO,TOTAL PATRIMONIO,INGRESOS OPERACIONALES,OTROS INGRESOS,COSTOS Y GASTOS OPERACIONALES,OTROS GASTOS,UTILIDAD (PERDIDA) OPERACIONAL,PROVISION PARA IMPUESTOS,UTILIDAD (PERDIDA) NETA,TIPO_EMPRESA
0,FAST COLOMBIA,2019,1506115534,2.504477e+08,0,1300928679,429575146,0.0,8.210270e+06,0,2.605942e+07,765247109,0,894237424,0,-128990315,38923731,-163921892,COMERCIAL
1,FAST COLOMBIA,2020,1529341118,1.234885e+08,0,1643112114,359235930,0.0,8.210270e+06,0,-3.495184e+08,410841381,0,718934555,0,-308093174,0,-398843699,COMERCIAL
2,FAST COLOMBIA,2020.0,1529341118,1.234885e+08,0,1643112114,359235930,0.0,8.210270e+06,0,-3.495184e+08,410841381,0,718934555,0,-308093174,0,-398843699,COMERCIAL
3,FAST COLOMBIA,2021.0,-2147483648,3.072363e+08,0,-2147483648,679566639,0.0,8.210270e+06,0,-3.994280e+08,1021983263,0,972390576,0,49592687,57984306,6402712,COMERCIAL
4,4- EMPRESA AEREA DE SERVICIOS Y FACILITACIÓN L...,2022,-2147483648,-2.147484e+09,0,-2147483648,-2147483648,0.0,-2.147484e+09,0,-2.147484e+09,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,COMERCIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,TAMPA,2016,1472529000,1.006761e+09,371469000,851424000,605739000,0.0,1.809820e+08,187395000,7.224220e+08,1177487000,23333000,978858000,51181000,198629000,27181000,143600000,CARGA
1333,TAMPA,2016.0,1472529000,1.006761e+09,371469000,851424000,605739000,0.0,1.809820e+08,187395000,7.224220e+08,1177487000,23333000,978858000,51181000,198629000,27181000,143600000,CARGA
1334,TAMPA,2017,1314657064,1.381439e+09,408098750,758864483,647622717,0.0,1.809818e+08,240164148,7.767482e+08,1049737310,26438850,933718867,51242445,116018443,32091893,59122955,CARGA
1335,TAMPA,2017.0,1314657064,1.386904e+09,406277144,758864483,647622717,0.0,1.809820e+08,184368894,7.767484e+08,1049737310,26438850,933718867,51242445,116018443,32091893,59122955,CARGA


In [167]:
# Define la ruta base y la carpeta de salida
base_path = Path().resolve().parent
folder = base_path / "output"

In [168]:
# Define el archivo de salida
output_file = folder / "df_consolidado.csv"

# Exporta el DataFrame a CSV
df_consolidado.to_csv(output_file, index=False, encoding="utf-8")
print(f"Archivo exportado exitosamente a: {output_file}")

Archivo exportado exitosamente a: C:\Users\andre\OneDrive\Documentos\Repositories\Aeronautica\modelo_riesgo\output\df_consolidado.csv
